# Pattern discovery

Suppose we have **100 sequences** of **length L** and a simple 
**pattern s** of **length k**, for k ≤ L. 
Assume it has N<sub>s</sub> occurrences in the sequences. 

Let **S** be the set of |Σ|k **possible patterns**, i.e. strings 
of length k.
The number of possible occurencies of s in each sequence 
is L − k + 1, therefore the total potential 
occurrences will be **100(L − k + 1)**. 

We want to be able to decide whether the **number of 
occurrences N<sub>s</sub>** observed for pattern s is something 
that we would expect to happen if the sequences were 
drawn under random (background) conditions.

##### P-value
P-value of 0.05. If we do 100 experiments, in each drawing 
100 random sequences, we would expect to see s occur in at 
least N<sub>s</sub> sequences just in 5 experiments.

##### Z-score
It is the number of standard deviations by which the observed 
value N<sub>s</sub> differs from the expected value. Since it is 
normalized it is suitable for comparing different motifs.

![zscore](images/zscore.png)

E(N<sub>s</sub>) and σ(N<sub>s</sub>)<sup>2</sup> be mean 
and variance of the number of occurrences of the pattern s
in the sequences generated according to the background 
distribution.

In [1]:
from Bio import SeqIO
import re
from Bio.Blast import NCBIXML
import numpy as np
import copy

### Background distribution
Count all possible patterns (background distribution) in human of a given lenght, 
no gaps, no ambiguous groups

In [5]:
# Load all human sequences into a list
seq_records = SeqIO.parse("data/human_up000005640.fasta", "fasta")

pattern_length = 6

S = 0
c = 0
for record in seq_records:
    c += 1
    S += len(record.seq) - pattern_length + 1

print("Number of proteins {}".format(c))
print("Number of possible patterns of length {}: {:,}".format(pattern_length, S))


Number of proteins 20371
Number of possible patterns of length 6: 11,257,512


### Simple pattern

Enumerate all possible patterns in human (background distribution).
The pattern is simple and has fixed lenght, no gaps, no ambiguous groups

In [6]:
patterns = {}
seq_records = list(SeqIO.parse("data/human_up000005640.fasta", "fasta"))
for record in seq_records:
    seq = str(record.seq)
    for i in range(len(seq) - pattern_length + 1):
        pattern = seq[i:i+pattern_length]
        patterns.setdefault(pattern, 0)
        patterns[pattern] += 1
        
patterns = sorted([(pattern, count) for pattern, count in patterns.items()], 
                  key=lambda x:x[1], reverse=True)

print("Number of observed patterns of length {}: {:,}".format(pattern_length, S))

Number of observed patterns of length 6: 11,257,512


In [7]:
expected = np.mean([count for pattern, count in patterns])
std = np.std([count for pattern, count in patterns])

print("Mean occurences {:.3}\nSTD {:.3}".format(expected, std))

# Calculate Z-score
for pattern, count in patterns[:10]:
    print("{} {:,} {:.3}".format(pattern, count, (count - expected) / std))


Mean occurences 1.42
STD 2.66
HTGEKP 2,613 9.83e+02
TGEKPY 2,188 8.23e+02
EEEEEE 1,656 6.23e+02
AAAAAA 1,559 5.86e+02
IHTGEK 1,477 5.55e+02
PPPPPP 1,415 5.32e+02
ECGKAF 1,412 5.31e+02
QQQQQQ 1,265 4.76e+02
SSSSSS 1,167 4.39e+02
EKPYKC 1,127 4.24e+02


### Gapped pattern

Calculate all possible patterns in human of a given lenght, 
with a maximum number of gaps (ex. 2), no ambiguous chars
```
ABCDEFG
ABC-EFG
AB--EFG
-BC-EFG
```
> WARNING 3 Gb of RAM (~10 minutes)


In [8]:
# Recursive function. Create a new tree level for increasing no. of gaps
def add_gaps(pattern, patterns, iteration, max_iterations):
    if iteration < max_iterations:
        for i in range(len(pattern)):
            new_pattern = pattern[:i] + "-" + pattern[i+1:]
            if new_pattern not in patterns:
                patterns.add(new_pattern)
                add_gaps(new_pattern, patterns, iteration + 1, max_iterations)
    return patterns

pattern_length = 6
max_gaps = 2

gapped_patterns = {}
for record in seq_records:
    seq = str(record.seq)
    for i in range(len(seq) - pattern_length + 1):
        pattern = seq[i:i+pattern_length]
        seq_patterns = add_gaps(pattern, set(), 0, max_gaps)
        for p in seq_patterns:
#             print(p)
            gapped_patterns.setdefault(p, 0)
            gapped_patterns[p] += 1

gapped_patterns = sorted([(pattern, c) for pattern, c in gapped_patterns.items()], 
                         key=lambda x:x[1], reverse=True)


In [9]:
expected = np.mean([count for pattern, count in gapped_patterns])
std = np.std([count for pattern, count in gapped_patterns])

print("Mean occurences {:.3}\nSTD {:.3}".format(expected, std))

# Calculate Z-score
c = 0
for pattern, count in gapped_patterns:
    # Skip patterns made of gaps and just one AA
    if len(set(pattern)) > 2:
        c += 1
        print("{} {:,} {:.3}".format(pattern, count, (count - expected) / std))
        if c >= 10:
           break


Mean occurences 14.3
STD 41.5
C--CGK 4,487 1.08e+02
-CGK-F 4,272 1.03e+02
CGK-F- 4,270 1.03e+02
H-GE-P 3,754 90.2
HTG--P 3,647 87.6
-HTGE- 3,542 85.1
--HTGE 3,541 85.0
HTGE-- 3,531 84.8
H-G-KP 3,529 84.7
-H-GEK 3,463 83.1


In [10]:
gapped_patterns = {}
for record in seq_records:
    seq = str(record.seq)
    for i in range(len(seq) - pattern_length + 1):
        pattern = seq[i:i+pattern_length]
        seq_patterns = add_gaps(pattern, set(), 0, max_gaps)
        for p in seq_patterns:
            print(p)
        break
    break
            

M-GP-D
-NGPV-
M-G-VD
M-GPV-
-NGP-D
-NGPVD
M--PVD
MN-PV-
MNGP--
MNGP-D
MNG--D
MN-P-D
MNGPV-
-N-PVD
MN--VD
MNG-VD
-NG-VD
MNG-V-
M-GPVD
--GPVD
MN-PVD


In [15]:
# Try with a set of functionally related sequences --> SwissProt proteins containing the N-glycosilation keyword
# seq_records = list(SeqIO.parse("data/sp_nglycosylation.fasta", "fasta"))

# Try with ELM instances. ELM motif .(N)[^P][ST]..
seq_records = list(SeqIO.parse("data/elm_nglycosylation.fasta", "fasta"))

# Recursive function. Create a new tree level for increasing no. of gaps
def add_gaps(pattern, patterns, iteration, max_iterations):
    if iteration < max_iterations:
        for i in range(len(pattern)):
            new_pattern = pattern[:i] + "-" + pattern[i+1:]
            if new_pattern not in patterns:
                patterns.add(new_pattern)
                add_gaps(new_pattern, patterns, iteration + 1, max_iterations)
    return patterns

pattern_length = 4
max_gaps = 2

gapped_patterns = {}
for record in seq_records:
    seq = str(record.seq)
    for i in range(len(seq) - pattern_length):
        pattern = seq[i:i+pattern_length]
        seq_patterns = add_gaps(pattern, set(), 0, max_gaps)
        for p in seq_patterns:
#             print(p)
            gapped_patterns.setdefault(p, 0)
            gapped_patterns[p] += 1

gapped_patterns = sorted([(pattern, c) for pattern, c in gapped_patterns.items()],
                         key=lambda x:x[1], reverse=True)


In [18]:
expected = np.mean([count for pattern, count in gapped_patterns])
std = np.std([count for pattern, count in gapped_patterns])

print("Mean occurences {:.3}\nSTD {:.3}".format(expected, std))

# Calculate Z-score
c = 0
for pattern, count in gapped_patterns:
    # Skip patterns made of gaps and just one AA
    if len(set(pattern)) > 2:
        c += 1
        print("{} {:,} {:.3}".format(pattern, count, (count - expected) / std))
        if c >= 10:
           break

Mean occurences 9.82
STD 23.5
-A-L 237 9.69
A-L- 237 9.69
-LA- 234 9.56
LA-- 234 9.56
--LA 233 9.52
-L-G 229 9.35
L-G- 229 9.35
--SL 228 9.3
-SL- 228 9.3
SL-- 228 9.3


### Varible length gaps
Calculate all possible patterns in human with a fixed number of gaps 
of variable length (ex. max length = 4), no ambiguous chars

```
A--B----C
A-BC
```

> WARNING it consumes xx Gb of RAM (~xx minutes)


In [24]:

number_gaps = 2
max_gap_len = 2

# Depth-first tree traversal
def gap_tree(gap_level, gaps_dict, gaps, number_gaps, max_gap_len):
    if gap_level < number_gaps:
        for i in range(max_gap_len + 1):
            gaps_dict[gap_level] = i
            if number_gaps - 1 in gaps_dict:  # Skip the incomplete branch
#                 print(gaps_dict)
                gaps.add(tuple(v for k, v in gaps_dict.items()))
            gap_tree(gap_level + 1, gaps_dict, gaps, number_gaps, max_gap_len)
    return gaps

gaps = gap_tree(0, {}, set(), number_gaps, max_gap_len)
print(gaps)

{(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}


In [25]:
variable_patterns = {}
for record in seq_records:
    seq = str(record.seq)
    for i in range(len(seq)):
#         print(seq[i:i+number_gaps*max_gap_len+number_gaps])
        for gap_list in gaps:
            pattern = seq[i]
            for j, gap in enumerate(gap_list):
                if i + j + 1 + gap < len(seq):
                    pattern = pattern + "-" * gap + seq[i+j+1+gap]
#             print(pattern)
            variable_patterns.setdefault(pattern, 0)
            variable_patterns[pattern] += 1
#         break
#     break

variable_patterns = sorted([(pattern, c) for pattern, c in variable_patterns.items()], 
                         key=lambda x:x[1], reverse=True)

In [23]:
expected = np.mean([count for pattern, count in variable_patterns])
std = np.std([count for pattern, count in variable_patterns])

print("Mean occurences {:.3}\nSTD {:.3}".format(expected, std))

# Calculate Z-score
c = 0
for pattern, count in variable_patterns:
    # Skip patterns made of gaps and just one AA
    if len(set(pattern)) > 2:
        c += 1
        print("{:<20} {:<10,} {:.3}".format(pattern, count, (count - expected) / std))
        if c >= 10:
           break

Mean occurences 5.18
STD 10.3
G---L--L             243        23.1
A-LL                 237        22.5
A---L--L             231        21.9
L-GG                 229        21.7
V---L--L             226        21.4
A--L-L               223        21.1
L---V--V             222        21.0
L-AA                 221        21.0
L-SS                 221        21.0
S--L-L               221        21.0
